In [1]:
#fear emotion

#데이터 pandas로 읽기, 구문이 탭으로 구분되어있음.
import pandas as pd
train_fear_df = pd.read_csv("feartrainalldata.txt","\t") 
test_fear_df = pd.read_csv("feartestalldata.txt","\t")


#konlpy는 띄어쓰기 알고리즘과 정규화를 이용해 맞춤법이 틀린 문장을 고치고 형태소 분석과 품사를 태깅해주는 클래스를 제공함.
from konlpy.tag import Okt
okt_fear = Okt()
# Okt를 쓴 이유 : 사용한 데이터는 인터넷 댓글이다. 어느정도 띄어쓰기가 되어있는 상태이기에 Okt가 가장 유리하다.


def fear_tokenize(doc):
    return ['/'.join(t) for t in okt_fear.pos(doc, norm=True, stem=False)]#08.22 stem=false로 바꿔봄. 
    #'구분자'.join(list) => 리스트 값과 값 사이에 구분자를 넣어 문자열을 하나로 합침. ex) '_'.join(['a','b','c'])이면 "a_b_c"로 반환
    #pos(text) => 품사를 태깅한 상태로 명사를 변환, ex) 컴퓨터(x), 컵퓨터,noun(o)
    #norm => 정규화 ex) 그래욬ㅋㅋㅋ => 그래요
    #stem => 근어로 표현 ex) 그래요 => 그렇다

    
train_fear_df.isnull().any()
train_fear_df['document'] = train_fear_df['document'].fillna(''); #null값을 ''로 대체
test_fear_df.isnull().any()
test_fear_df['document'] = test_fear_df['document'].fillna('');
# 빈민자 처리

train_fear_docs = [(fear_tokenize(row[1]), row[2]) for row in train_fear_df.values] 
#traindata 저장 row[1] : document / row[2] : label
test_fear_docs = [(fear_tokenize(row[1]),row[2]) for row in test_fear_df.values] 
#testdata 저장
tokens = [t for d in train_fear_docs for t in d[0]]

import nltk
fear_text = nltk.Text(tokens, name='FEAR') 
#문서 하나를 편리하게 탐색할 수 있는 기능 제공 (vocab().most_common() 사용)
# print(len(fear_text.tokens)) #토큰 개수
# print(len(set(fear_text.tokens))) #중복을 제외한 토큰 수
# print(fear_text.vocab().most_common(10)) #출력빈도가 높은 상위 토큰 10개


#countvectorization
FEAR_FREQUENCY_COUNT = 3000; #자주 사용되는 토큰 설정(3000)이상 하면 Memory Error
fear_selected_words = [f[0] for f in fear_text.vocab().most_common(FEAR_FREQUENCY_COUNT)] #선택되어진 토큰들

def fear_term_frequency(doc):
    return [doc.count(word) for word in fear_selected_words]
#상위 3000개의 단어들이 문장 속에 몇 개 있는지 확인한다.
# ex) [0,0,0,0,1,0,1,0, ... 0,0,0] -> 이게 3000개
x_fear_train = [fear_term_frequency(d) for d,_ in train_fear_docs]
x_fear_test = [fear_term_frequency(d) for d,_ in test_fear_docs]
#x축에는 문서에 들어가는 단어 개수(단어들의 빈도수 정보)

y_fear_train = [c for _,c in train_fear_docs]
y_fear_test = [c for _,c in test_fear_docs]
#y축에는 1 or 0, 분류 결과


import numpy as np
x_fear_train = np.asarray(x_fear_train).astype('float32')
x_fear_test = np.asarray(x_fear_test). astype('float32')

y_fear_train = np.asarray(y_fear_train).astype('float32')
y_fear_test = np.asarray(y_fear_test).astype('float32')
#np.asarray는 np.array와 달리 데이터 형태가 같을 때 복사하지 않음.
#데이터 float로 형 변환
#asarray와 array의 차이


import tensorflow as tf #텐서플로 케라스
fear_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(FEAR_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
# relu를 쓴 이유 : 연산의 cost가 적고, 구현이 간단함.
# sigmoid를 쓴 이유 : 다중 감정 분석이지만 이진 분류 문제로 해결했기 때문에 출력층의 활성화함수는 이진 분류 문제에 효과적인 시그모이드 사용 
# relu층이 두 층인 이유 : 세 층으로 했을 때, testset에 대한 정확도가 떨어졌다. 단순히 하는 것이 overfitting을 막는 방법
# dense층이 64개의 유닛인 이유는?

    # 세 층으로 했을 때와 두 층으로 했을 때 정확도 비교 결과 두 층의 정확도가 더 높고
    # 세 층을 했을 때, overfitting?
#레이어 구성은 dense층은 64개의 유닛, 활성함수는 relu, 마지막 층은 sigmoid 활성화 함수 사용



fear_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )
#손실 함수는 binary_crossentropy, RMSprop 옵티마이저를 통해 경사하강법 진행
# binary_crossentropy 사용 이유
#label 값이 0 또는 1인 경우에는 즉, sigmoid 함수에서는 binary_crossentropy가 더 좋다. 

# RMSprop 옵티마이저 사용 이유
# Adam이 더 좋음


import matplotlib.pyplot as plt
fear_model.fit(x_fear_train, y_fear_train, epochs=50, batch_size=512)
#fear_callbacks = [EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)]
#배치 사이즈 줄이면 한 번에 판단하는 데이터 수 증가함. 에포크 50번
fear_results = fear_model.evaluate(x_fear_test, y_fear_test)


fear_review = "너무 무섭다"
fear_token = fear_tokenize(fear_review)

fear_tf = fear_term_frequency(fear_token)
f_data = np.expand_dims(np.asarray(fear_tf).astype('float32'),axis=0)
float(fear_model.predict(f_data))

Epoch 1/50
53/53 [==============================] - 0s 7ms/step - loss: 0.4443 - binary_accuracy: 0.8448
Epoch 2/50
53/53 [==============================] - 0s 8ms/step - loss: 0.3250 - binary_accuracy: 0.8668
Epoch 3/50
53/53 [==============================] - 0s 8ms/step - loss: 0.2869 - binary_accuracy: 0.8839
Epoch 4/50
53/53 [==============================] - 0s 7ms/step - loss: 0.2642 - binary_accuracy: 0.8936
Epoch 5/50
53/53 [==============================] - 0s 8ms/step - loss: 0.2446 - binary_accuracy: 0.9014
Epoch 6/50
53/53 [==============================] - 0s 6ms/step - loss: 0.2256 - binary_accuracy: 0.9104
Epoch 7/50
53/53 [==============================] - 0s 7ms/step - loss: 0.2070 - binary_accuracy: 0.9187
Epoch 8/50
53/53 [==============================] - 0s 7ms/step - loss: 0.1869 - binary_accuracy: 0.9284
Epoch 9/50
53/53 [==============================] - 0s 7ms/step - loss: 0.1672 - binary_accuracy: 0.9355
Epoch 10/50
53/53 [==============================] - 0s

0.9999767541885376

In [3]:
fear_model.save('fear_model.h5')

In [4]:
#surprise emotion

import pandas as pd
train_surprise_df = pd.read_csv("surprisetrainalldata.txt","\t")
test_surprise_df = pd.read_csv("surprisetestalldata.txt","\t")

from konlpy.tag import Okt
okt_surprise = Okt()

def surprise_tokenize(doc):
    return ['/'.join(t) for t in okt_surprise.pos(doc, norm=True, stem=False)]

train_surprise_df.isnull().any()
train_surprise_df['document'] = train_surprise_df['document'].fillna('');
test_surprise_df.isnull().any()
test_surprise_df['document'] = test_surprise_df['document'].fillna('');

train_surprise_docs = [(surprise_tokenize(row[1]), row[2]) for row in train_surprise_df.values]
test_surprise_docs = [(surprise_tokenize(row[1]),row[2]) for row in test_surprise_df.values]

tokens = [t for d in train_surprise_docs for t in d[0]]

import nltk
surprise_text = nltk.Text(tokens, name='SURP')
#print(len(surprise_text.tokens))
#print(len(set(surprise_text.tokens)))
#print(surprise_text.vocab().most_common(10))

SURPRISE_FREQUENCY_COUNT = 3000; #variable
surprise_selected_words = [f[0] for f in surprise_text.vocab().most_common(SURPRISE_FREQUENCY_COUNT)]

def surprise_term_frequency(doc):
    return [doc.count(word) for word in surprise_selected_words]

x_surprise_train = [surprise_term_frequency(d) for d,_ in train_surprise_docs]
x_surprise_test = [surprise_term_frequency(d) for d,_ in test_surprise_docs]
y_surprise_train = [c for _,c in train_surprise_docs]
y_surprise_test = [c for _,c in test_surprise_docs]

import numpy as np
x_surprise_train = np.asarray(x_surprise_train).astype('float32')
x_surprise_test = np.asarray(x_surprise_test). astype('float32')

y_surprise_train = np.asarray(y_surprise_train).astype('float32')
y_surprise_test = np.asarray(y_surprise_test).astype('float32')

import tensorflow as tf
surprise_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(SURPRISE_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

surprise_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

surprise_model.fit(x_surprise_train, y_surprise_train, epochs=50, batch_size=256)
surprise_results = surprise_model.evaluate(x_surprise_test, y_surprise_test)
surprise_review = "너무 놀랍다"
surprise_token = surprise_tokenize(surprise_review)

tf = surprise_term_frequency(surprise_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(surprise_model.predict(data))

Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.4091 - binary_accuracy: 0.8470
Epoch 2/50
106/106 [==============================] - 1s 5ms/step - loss: 0.3367 - binary_accuracy: 0.8591
Epoch 3/50
106/106 [==============================] - 1s 5ms/step - loss: 0.3147 - binary_accuracy: 0.8681
Epoch 4/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2988 - binary_accuracy: 0.8764
Epoch 5/50
106/106 [==============================] - 1s 6ms/step - loss: 0.2831 - binary_accuracy: 0.8839
Epoch 6/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2654 - binary_accuracy: 0.8930
Epoch 7/50
106/106 [==============================] - 1s 6ms/step - loss: 0.2485 - binary_accuracy: 0.9000
Epoch 8/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2301 - binary_accuracy: 0.9081
Epoch 9/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2129 - binary_accuracy: 0.9165
Epoch 10/50
106/106 [================

0.0034102643840014935

In [5]:
surprise_model.save('surprise_model.h5')

In [6]:
#anger emotion

import pandas as pd
train_anger_df = pd.read_csv("angertrainalldata.txt","\t")
test_anger_df = pd.read_csv("angertestalldata.txt","\t")

from konlpy.tag import Okt
okt_anger = Okt()

def anger_tokenize(doc):
    return ['/'.join(t) for t in okt_anger.pos(doc, norm=True, stem=False)]

train_anger_df.isnull().any()
train_anger_df['document'] = train_anger_df['document'].fillna('');
test_anger_df.isnull().any()
test_anger_df['document'] = test_anger_df['document'].fillna('');

train_anger_docs = [(anger_tokenize(row[1]), row[2]) for row in train_anger_df.values]
test_anger_docs = [(anger_tokenize(row[1]),row[2]) for row in test_anger_df.values]

tokens = [t for d in train_anger_docs for t in d[0]]

import nltk
anger_text = nltk.Text(tokens, name='ANGE')
#print(len(anger_text.tokens))
#print(len(set(anger_text.tokens)))
#print(anger_text.vocab().most_common(10))

ANGER_FREQUENCY_COUNT = 3000; #variable
anger_selected_words = [f[0] for f in anger_text.vocab().most_common(ANGER_FREQUENCY_COUNT)]

def anger_term_frequency(doc):
    return [doc.count(word) for word in anger_selected_words]

x_anger_train = [anger_term_frequency(d) for d,_ in train_anger_docs]
x_anger_test = [anger_term_frequency(d) for d,_ in test_anger_docs]
y_anger_train = [c for _,c in train_anger_docs]
y_anger_test = [c for _,c in test_anger_docs]

import numpy as np
x_anger_train = np.asarray(x_anger_train).astype('float32')
x_anger_test = np.asarray(x_anger_test). astype('float32')

y_anger_train = np.asarray(y_anger_train).astype('float32')
y_anger_test = np.asarray(y_anger_test).astype('float32')

import tensorflow as tf
anger_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(ANGER_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

anger_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

anger_model.fit(x_anger_train, y_anger_train, epochs=50, batch_size=256)
anger_results = anger_model.evaluate(x_anger_test, y_anger_test)

anger_review = "너무 화난다"
anger_token = anger_tokenize(anger_review)

tf = anger_term_frequency(anger_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(anger_model.predict(data))

Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.4055 - binary_accuracy: 0.8510
Epoch 2/50
106/106 [==============================] - 0s 4ms/step - loss: 0.3140 - binary_accuracy: 0.8687
Epoch 3/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2903 - binary_accuracy: 0.8799
Epoch 4/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2707 - binary_accuracy: 0.8890
Epoch 5/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2504 - binary_accuracy: 0.8994
Epoch 6/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2305 - binary_accuracy: 0.9092
Epoch 7/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2099 - binary_accuracy: 0.9181
Epoch 8/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1894 - binary_accuracy: 0.9280
Epoch 9/50
106/106 [==============================] - 0s 5ms/step - loss: 0.1697 - binary_accuracy: 0.9362
Epoch 10/50
106/106 [================

0.04778598994016647

In [7]:
anger_model.save('anger_model.h5')

In [8]:
#sadness emotion

import pandas as pd
train_sadness_df = pd.read_csv("sadnesstrainalldata.txt","\t")
test_sadness_df = pd.read_csv("sadnesstestalldata.txt","\t")

from konlpy.tag import Okt
okt_sadness = Okt()

def sadness_tokenize(doc):
    return ['/'.join(t) for t in okt_sadness.pos(doc, norm=True, stem=False)]

train_sadness_df.isnull().any()
train_sadness_df['document'] = train_sadness_df['document'].fillna('');
test_sadness_df.isnull().any()
test_sadness_df['document'] = test_sadness_df['document'].fillna('');

train_sadness_docs = [(sadness_tokenize(row[1]), row[2]) for row in train_sadness_df.values]
test_sadness_docs = [(sadness_tokenize(row[1]),row[2]) for row in test_sadness_df.values]

tokens = [t for d in train_sadness_docs for t in d[0]]

import nltk
sadness_text = nltk.Text(tokens, name='SADN')
#print(len(sadness_text.tokens))
#print(len(set(sadness_text.tokens)))
#print(sadness_text.vocab().most_common(10))

SADNESS_FREQUENCY_COUNT = 3000; #variable
sadness_selected_words = [f[0] for f in fear_text.vocab().most_common(SADNESS_FREQUENCY_COUNT)]

def sadness_term_frequency(doc):
    return [doc.count(word) for word in sadness_selected_words]

x_sadness_train = [sadness_term_frequency(d) for d,_ in train_sadness_docs]
x_sadness_test = [sadness_term_frequency(d) for d,_ in test_sadness_docs]
y_sadness_train = [c for _,c in train_sadness_docs]
y_sadness_test = [c for _,c in test_sadness_docs]

import numpy as np
x_sadness_train = np.asarray(x_sadness_train).astype('float32')
x_sadness_test = np.asarray(x_sadness_test). astype('float32')

y_sadness_train = np.asarray(y_sadness_train).astype('float32')
y_sadness_test = np.asarray(y_sadness_test).astype('float32')

import tensorflow as tf
sadness_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(SADNESS_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

sadness_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

sadness_model.fit(x_sadness_train, y_sadness_train, epochs=50, batch_size=256)
sadness_results = sadness_model.evaluate(x_sadness_test, y_sadness_test)
sadness_review = "너무 화난다"
sadness_token = sadness_tokenize(sadness_review)

tf = sadness_term_frequency(sadness_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(sadness_model.predict(data))


Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.3808 - binary_accuracy: 0.8598
Epoch 2/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2805 - binary_accuracy: 0.8880
Epoch 3/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2550 - binary_accuracy: 0.8982
Epoch 4/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2355 - binary_accuracy: 0.9071
Epoch 5/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2186 - binary_accuracy: 0.9152
Epoch 6/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2023 - binary_accuracy: 0.9218
Epoch 7/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1865 - binary_accuracy: 0.9291
Epoch 8/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1706 - binary_accuracy: 0.9364
Epoch 9/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1556 - binary_accuracy: 0.9423
Epoch 10/50
106/106 [================

0.358499675989151

In [9]:
sadness_model.save('sadness_model.h5')

In [10]:
#neutral emotion

import pandas as pd
train_neutral_df = pd.read_csv("neutraltrainalldata.txt","\t")
test_neutral_df = pd.read_csv("neutraltestalldata.txt","\t")

from konlpy.tag import Okt
okt_neutral = Okt()

def neutral_tokenize(doc):
    return ['/'.join(t) for t in okt_neutral.pos(doc, norm=True, stem=False)]

train_neutral_df.isnull().any()
train_neutral_df['document'] = train_neutral_df['document'].fillna('');
test_neutral_df.isnull().any()
test_neutral_df['document'] = test_neutral_df['document'].fillna('');

train_neutral_docs = [(neutral_tokenize(row[1]), row[2]) for row in train_neutral_df.values]
test_neutral_docs = [(neutral_tokenize(row[1]),row[2]) for row in test_neutral_df.values]

tokens = [t for d in train_neutral_docs for t in d[0]]

import nltk
neutral_text = nltk.Text(tokens, name='NEUT')
#print(len(neutral_text.tokens))
#print(len(set(neutral_text.tokens)))
#print(neutral_text.vocab().most_common(10))

NEUTRAL_FREQUENCY_COUNT = 3000; #variable
neutral_selected_words = [f[0] for f in neutral_text.vocab().most_common(NEUTRAL_FREQUENCY_COUNT)]

def neutral_term_frequency(doc):
    return [doc.count(word) for word in neutral_selected_words]

x_neutral_train = [neutral_term_frequency(d) for d,_ in train_neutral_docs]
x_neutral_test = [neutral_term_frequency(d) for d,_ in test_neutral_docs]
y_neutral_train = [c for _,c in train_neutral_docs]
y_neutral_test = [c for _,c in test_neutral_docs]

import numpy as np
x_neutral_train = np.asarray(x_neutral_train).astype('float32')
x_neutral_test = np.asarray(x_neutral_test). astype('float32')

y_neutral_train = np.asarray(y_neutral_train).astype('float32')
y_neutral_test = np.asarray(y_neutral_test).astype('float32')

import tensorflow as tf
neutral_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(NEUTRAL_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

neutral_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

neutral_model.fit(x_neutral_train, y_neutral_train, epochs=50, batch_size=256)
neutral_results = neutral_model.evaluate(x_neutral_test, y_neutral_test)
neutral_review = "너무 화난다"
neutral_token = neutral_tokenize(neutral_review)

tf = neutral_term_frequency(neutral_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(neutral_model.predict(data))

Epoch 1/50
106/106 [==============================] - 0s 5ms/step - loss: 0.3941 - binary_accuracy: 0.8723
Epoch 2/50
106/106 [==============================] - 0s 4ms/step - loss: 0.3272 - binary_accuracy: 0.8751
Epoch 3/50
106/106 [==============================] - 0s 4ms/step - loss: 0.3084 - binary_accuracy: 0.8782
Epoch 4/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2920 - binary_accuracy: 0.8838
Epoch 5/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2737 - binary_accuracy: 0.8913
Epoch 6/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2518 - binary_accuracy: 0.9022
Epoch 7/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2283 - binary_accuracy: 0.9113
Epoch 8/50
106/106 [==============================] - 0s 4ms/step - loss: 0.2044 - binary_accuracy: 0.9215
Epoch 9/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1811 - binary_accuracy: 0.9324
Epoch 10/50
106/106 [================

0.002875301754102111

In [11]:
neutral_model.save('neutral_model.h5')

In [12]:
# happy emotion

import pandas as pd
train_happy_df = pd.read_csv("happytrainalldata.txt","\t")
test_happy_df = pd.read_csv("happytestalldata.txt","\t")

from konlpy.tag import Okt
okt_happy = Okt()

def happy_tokenize(doc):
    return ['/'.join(t) for t in okt_happy.pos(doc, norm=True, stem=False)]

train_happy_df.isnull().any()
train_happy_df['document'] = train_happy_df['document'].fillna('');
test_happy_df.isnull().any()
test_happy_df['document'] = test_happy_df['document'].fillna('');

train_happy_docs = [(happy_tokenize(row[1]), row[2]) for row in train_happy_df.values]
test_happy_docs = [(happy_tokenize(row[1]),row[2]) for row in test_happy_df.values]

tokens = [t for d in train_happy_docs for t in d[0]]

import nltk
happy_text = nltk.Text(tokens, name='HAPP')
#print(len(happy_text.tokens))
#print(len(set(happy_text.tokens)))
#print(happy_text.vocab().most_common(10))

HAPPY_FREQUENCY_COUNT = 3000; #variable
happy_selected_words = [f[0] for f in happy_text.vocab().most_common(HAPPY_FREQUENCY_COUNT)]

def happy_term_frequency(doc):
    return [doc.count(word) for word in happy_selected_words]

x_happy_train = [happy_term_frequency(d) for d,_ in train_happy_docs]
x_happy_test = [happy_term_frequency(d) for d,_ in test_happy_docs]
y_happy_train = [c for _,c in train_happy_docs]
y_happy_test = [c for _,c in test_happy_docs]

import numpy as np
x_happy_train = np.asarray(x_happy_train).astype('float32')
x_happy_test = np.asarray(x_happy_test). astype('float32')

y_happy_train = np.asarray(y_happy_train).astype('float32')
y_happy_test = np.asarray(y_happy_test).astype('float32')

import tensorflow as tf
happy_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(HAPPY_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
happy_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )

happy_model.fit(x_happy_train, y_happy_train, epochs=50, batch_size=256)
happy_results = happy_model.evaluate(x_happy_test, y_happy_test)

happy_review = "너무 화난다"
happy_token = happy_tokenize(happy_review)

tf = happy_term_frequency(happy_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(happy_model.predict(data))

Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.3435 - binary_accuracy: 0.8746A: 0s - loss: 0.3973 - binary_accuracy: 0
Epoch 2/50
106/106 [==============================] - 0s 5ms/step - loss: 0.2150 - binary_accuracy: 0.9230
Epoch 3/50
106/106 [==============================] - 1s 5ms/step - loss: 0.1881 - binary_accuracy: 0.9311
Epoch 4/50
106/106 [==============================] - 0s 5ms/step - loss: 0.1730 - binary_accuracy: 0.9373
Epoch 5/50
106/106 [==============================] - 1s 5ms/step - loss: 0.1596 - binary_accuracy: 0.9412
Epoch 6/50
106/106 [==============================] - 0s 5ms/step - loss: 0.1482 - binary_accuracy: 0.9452
Epoch 7/50
106/106 [==============================] - 1s 5ms/step - loss: 0.1372 - binary_accuracy: 0.9494
Epoch 8/50
106/106 [==============================] - 0s 5ms/step - loss: 0.1260 - binary_accuracy: 0.9539
Epoch 9/50
106/106 [==============================] - 1s 5ms/step - loss: 0.1154 - binary_accuracy: 0.9

0.3424101769924164

In [13]:
happy_model.save('happy_model.h5')

In [14]:
#disgust emotion

import pandas as pd
train_disgust_df = pd.read_csv("disgusttrainalldata.txt","\t")
test_disgust_df = pd.read_csv("disgusttestalldata.txt","\t")

from konlpy.tag import Okt
okt_disgust = Okt()

def disgust_tokenize(doc):
    return ['/'.join(t) for t in okt_disgust.pos(doc, norm=True, stem=False)]

train_disgust_df.isnull().any()
train_disgust_df['document'] = train_disgust_df['document'].fillna('');
test_disgust_df.isnull().any()
test_disgust_df['document'] = test_disgust_df['document'].fillna('');

train_disgust_docs = [(disgust_tokenize(row[1]), row[2]) for row in train_disgust_df.values]
test_disgust_docs = [(disgust_tokenize(row[1]),row[2]) for row in test_disgust_df.values]

tokens = [t for d in train_disgust_docs for t in d[0]]

import nltk
disgust_text = nltk.Text(tokens, name='DISG')
#print(len(disgust_text.tokens))
#print(len(set(disgust_text.tokens)))
#print(disgust_text.vocab().most_common(10))

DISGUST_FREQUENCY_COUNT = 3000; #variable
disgust_selected_words = [f[0] for f in disgust_text.vocab().most_common(DISGUST_FREQUENCY_COUNT)]

def disgust_term_frequency(doc):
    return [doc.count(word) for word in disgust_selected_words]

x_disgust_train = [disgust_term_frequency(d) for d,_ in train_disgust_docs]
x_disgust_test = [disgust_term_frequency(d) for d,_ in test_disgust_docs]
y_disgust_train = [c for _,c in train_disgust_docs]
y_disgust_test = [c for _,c in test_disgust_docs]

import numpy as np
x_disgust_train = np.asarray(x_disgust_train).astype('float32')
x_disgust_test = np.asarray(x_disgust_test). astype('float32')

y_disgust_train = np.asarray(y_disgust_train).astype('float32')
y_disgust_test = np.asarray(y_disgust_test).astype('float32')

import tensorflow as tf
disgust_model= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu',input_shape=(DISGUST_FREQUENCY_COUNT,)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
disgust_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=[tf.keras.metrics.binary_accuracy]
             )
disgust_model.fit(x_disgust_train, y_disgust_train, epochs=50, batch_size=256)
disgust_results = neutral_model.evaluate(x_disgust_test, y_disgust_test)
disgust_review = "너무 화난다"
disgust_token = disgust_tokenize(disgust_review)

tf = disgust_term_frequency(disgust_token)
data = np.expand_dims(np.asarray(tf).astype('float32'),axis=0)
float(disgust_model.predict(data))


Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.4165 - binary_accuracy: 0.8585
Epoch 2/50
106/106 [==============================] - 0s 5ms/step - loss: 0.3465 - binary_accuracy: 0.8598
Epoch 3/50
106/106 [==============================] - 1s 5ms/step - loss: 0.3271 - binary_accuracy: 0.8653
Epoch 4/50
106/106 [==============================] - 0s 5ms/step - loss: 0.3095 - binary_accuracy: 0.8750
Epoch 5/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2890 - binary_accuracy: 0.8843
Epoch 6/50
106/106 [==============================] - 0s 5ms/step - loss: 0.2652 - binary_accuracy: 0.8964
Epoch 7/50
106/106 [==============================] - 1s 5ms/step - loss: 0.2384 - binary_accuracy: 0.9075
Epoch 8/50
106/106 [==============================] - 0s 5ms/step - loss: 0.2121 - binary_accuracy: 0.9204
Epoch 9/50
106/106 [==============================] - 1s 5ms/step - loss: 0.1859 - binary_accuracy: 0.9319
Epoch 10/50
106/106 [================

0.016523953527212143

In [15]:
disgust_model.save('disgust_model.h5')